In [ ]:
!nvcc --version

In [ ]:
!pip install catboost

In [ ]:
!pip install --upgrade catboost


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
data = pd.read_csv('./train_preprocessing_clean.csv', delimiter=';')

In [ ]:
data.isna().sum()

In [ ]:
X=data.drop(['Month 4'],axis=1)

In [ ]:
y=data['Month 4']

In [ ]:
categorical_columns = ['Cluster','Reference proxy','Product Life cycel status','Product  Line proxy', 'Division proxy', 'Customer Persona proxy', 'Strategic Product Family proxy','Date']

In [ ]:
categorical_features_indices = [X.columns.get_loc(col) for col in categorical_columns]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)

In [ ]:

#hyperparamètres à rechercher
param_grid = {
    'iterations': [300, 1000],
    'learning_rate': [0.05, 0.1],
    'depth': [8, 10]
}

# Créer une instance du modèle CatBoostRegressor
base_model = CatBoostRegressor(loss_function='RMSE', random_seed=42, task_type='GPU')

# Créer l'objet GridSearchCV
grid_search = GridSearchCV(estimator=base_model, param_grid=param_grid, cv=3, scoring='neg_root_mean_squared_error')

# Effectuer la recherche sur grille sur les données d'entraînement
grid_result = grid_search.fit(X_train, y_train, cat_features=categorical_features_indices)

# Afficher les meilleurs paramètres trouvés
print("Meilleurs paramètres:", grid_result.best_params_)


In [ ]:
grid_result.best_params_

In [ ]:
# Créez un modèle CatBoostRegressor
model = CatBoostRegressor(iterations=1000,
                          learning_rate=0.1,  # Taux d'apprentissage
                          depth=12,  # Profondeur maximale des arbres
                          loss_function='RMSE',  # Fonction de perte (Root Mean Squared Error pour la régression)
                          random_seed=42, task_type='GPU')  # Graine aléatoire pour la reproductibilité

# Entraînez le modèle sur les données d'entraînement
model.fit(X_train, y_train, cat_features=categorical_features_indices)


In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[predictions<0] = 0

In [ ]:
predictions = predictions.astype(int)

In [ ]:
predictions

In [ ]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

In [ ]:
R_0= np.sqrt(mean_squared_error(y_test,[0]*len(y_test)))
RMSE_pred=np.sqrt(mean_squared_error(y_test,predictions))

In [ ]:
Score_hfactory=(R_0 - 0.8 * RMSE_pred)/R_0

In [ ]:
Score_hfactory

In [ ]:
X_valid = pd.read_csv('./test_preprocessing_clean.csv', delimiter=';')

In [ ]:
X_valid.isna().sum()

In [ ]:
y_valid=model.predict(X_valid)

In [ ]:
y_valid[y_valid<0]=0

In [ ]:
y_valid = y_valid.astype(int)

In [ ]:
y_valid

In [ ]:
to_submit = pd.read_csv('./Datasets/y_test_example.csv', delimiter=';')

In [ ]:
to_submit['Month 4'] = y_valid

In [ ]:
to_submit

In [ ]:
to_submit.to_csv('y_test_9.csv', index=False, sep=';')